In [4]:
%load_ext autoreload
%autoreload 2

In [6]:
import requests

url = 'https://mvic.sos.state.mi.us/Voter/GetMvicBallot/1828/679/'

response = requests.get(url,
            headers={
                'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Geck o/20100101 Firefox/40.1'
            },
            verify=False,
)

response.status_code

/Users/Browning/Documents/citizenlabs/elections-api/.venv/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


200

In [7]:
html = response.text.strip()

In [10]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html)

In [46]:
ballot = soup.find(id='PreviewMvicBallot').div.div.find_all('div', recursive=False)[1]

In [49]:
data = {}

In [66]:
for index, section in enumerate(ballot.find_all('div', recursive=False)):
    #section = section.div
    
    for div in section.find_all('div', {"class": "section"}):
        print(div.prettify())
    
    
    
    break

<div class="col-md-12 section" valign="top">
 NONPARTISAN SECTION
</div>


In [105]:
data = {}
section = None
division = None
office = None



offices = ballot.find(id='generalElectionOffices')


for index, item in enumerate(offices.find_all('div', {"class": ["section", "division", "office", "term", "candidate"]}), start=1):

    print((index, item['class'], item.text))
    
    if "section" in item['class']:
        section = {}
        label = item.text
        data[label] = section
        
    elif "division" in item['class']:
        label = item.text.replace(' - Continued', '')
        
        try:
            division = section[label]
        except KeyError:
            division = {}
            
        section[label] = division
        
    elif "office" in item['class']:
        label = item.text
        
        try:
            office = division[label]
        except KeyError:
            office = {'term': [], 'candidates': []}
            
        division[label] = office
        
    elif "term" in item['class']:
        office['term'].append(item.text)
        
    elif "candidate" in item['class']:
        office['candidates'].append(item.text)
        
    
        

from pprint import pprint
print()
pprint(data)

(1, ['col-md-12', 'section'], 'NONPARTISAN SECTION')
(2, ['col-md-12', 'division'], 'CITY')
(3, ['col-md-12', 'office'], 'MAYOR')
(4, ['col-md-12', 'term'], 'Vote for not more than 1')
(5, ['col-xs-7', 'candidate'], 'Rosalynn Bliss')
(6, ['col-xs-7', 'candidate'], 'Daniel Allen Schutte')
(7, ['division', 'col-xs-12', 'd-none', 'd-md-block', 'd-sm-none'], 'CITY - Continued')
(8, ['col-md-12', 'office'], 'COMMISSIONER BY WARD')
(9, ['col-md-12', 'term'], 'WARD 1')
(10, ['col-md-12', 'term'], 'Vote for not more than 1')
(11, ['col-xs-7', 'candidate'], 'Allison Kay Lutz')
(12, ['col-xs-7', 'candidate'], "Jon O'Connor")
(13, ['division', 'col-xs-12', 'd-none', 'd-md-block', 'd-sm-none'], 'CITY - Continued')
(14, ['col-md-12', 'office'], 'LIBRARY BOARD DIRECTOR')
(15, ['col-md-12', 'term'], 'Vote for not more than 1')
(16, ['col-xs-7', 'candidate'], 'Kathryn Dilley')
(17, ['col-xs-7', 'candidate'], 'Jonathan Helder')
(18, ['col-md-12', 'office'], 'LIBRARY BOARD DIRECTOR')
(19, ['col-md-12', 

In [109]:
data = {}
section = None
division = None
office = None



proposals = ballot.find(id='proposals')


for index, item in enumerate(proposals.find_all('div', {"class": ["section", "division", "proposalTitle", "proposalText"]}), start=1):

    print((index, item['class'], item.text))
    
    if "section" in item['class']:
        section = {}
        label = item.text
        data[label] = section
        
    elif "division" in item['class']:
        label = item.text.replace(' - Continued', '')
        
        try:
            division = section[label]
        except KeyError:
            division = {}
            
        section[label] = division
        
    elif "proposalTitle" in item['class']:
        label = item.text
        
        try:
            proposal = division[label]
        except KeyError:
            proposal = {'text': None}
            
        division[label] = proposal
        
    elif "proposalText" in item['class']:
        proposal['text'] = item.text
        

        
from pprint import pprint
print()
pprint(data)

(1, ['col-md-12', 'section'], 'PROPOSAL SECTION')
(2, ['col-md-12', 'division'], 'CITY PROPOSALS')
(3, ['col-md-12', 'proposalTitle'], 'City of Grand Rapids\nCity of Grand Rapids Charter Amendment authorizing the levy of a permanent tax millage of 1.25 mills for PARKS, POOLS AND PLAYGROUNDS commencing January 1, 2021. The current levy of 0.98 mills expires in 2020.')
(4, ['col-md-12', 'proposalText'], "\xa0\n\xa0\n\xa0\nIt is proposed that Section 18(e)(4) be added to the City Charter authorizing the levy of 1.25 mills commencing January 1, 2021, to be used for City-owned parks, pools, and playgrounds which will raise an estimated $5,000,000 in the levy's first year.\nA portion of the revenues received would also be disbursed to:\nBrownfield Redevelopment Authority, Downtown Development Authority, SmartZone Local Development Finance Authority, Monroe North TIFA, Southtown CIA, Michigan Street CIA. Westside CIA, North Quarter CIA, South Division/Grandville Avenue and Uptown CIA .\nShall